### data

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', '{:.3f}'.format)

In [99]:
def load_generation_data(include_files=None, date_filter=None):
    data_dir = "/Users/jangseohyun/Documents/workspace/symply/DER/data/generation"
    all_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.csv')])

    if include_files is not None:
        for file in include_files:
            if file not in all_files:
                raise ValueError(f"파일을 찾을 수 없습니다: {file}")
        files_to_load = [f for f in all_files if f in include_files]
    else:
        files_to_load = all_files

    I = len(files_to_load)  
    T = 24 
    generation_data = np.zeros((I, T)) 

    loaded_files = []
    
    for idx, file in enumerate(files_to_load):
        file_path = os.path.join(data_dir, file)
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip()

        date_col = "Date"
        hour_col = "Hour (Eastern Time, Daylight-Adjusted)"
        gen_col = "Electricity Generated"

        if any(col not in df.columns for col in [date_col, hour_col, gen_col]):
            print(f"{file}: 필요한 컬럼 없음. 스킵됨.")
            continue

        if date_filter:
            df = df[df[date_col] == date_filter]
            if df.empty:
                print(f"{file}: {date_filter} 데이터 없음. 스킵됨.")
                continue 

        df = df[df[hour_col].astype(str).str.match(r'^\d+$')]
        df["Time"] = df[hour_col].astype(int)
        df = df[df["Time"].between(0, 23)]

        for t in range(T):
            if t in df["Time"].values:
                generation_data[idx, t] = df[df["Time"] == t][gen_col].values[0]

        loaded_files.append(file)

    print(f"✅ 총 {I}개 파일을 불러왔습니다: {', '.join(loaded_files)}")

    return generation_data, I, T

def generate_randomized_generation(I, T, S, data, randomness_level):
    np.random.seed(7)

    noise_ranges = {
        "low": (0.8, 1.2),
        "medium": (0.5, 1.5),
        "high": (0.2, 1.8),
    }

    if randomness_level not in noise_ranges:
        raise ValueError("Invalid randomness level. Please choose 'low', 'medium', or 'high'.")

    low, high = noise_ranges[randomness_level]
    noise_factors = np.random.uniform(low, high, size=(I, T, S))

    generation_r = np.expand_dims(data, axis=-1) * noise_factors
    
    print(f"📊 데이터 Shape: I={I}, T={T}, S={S}")
    return generation_r

def plot_generation_data(generation_data, I):
    hours = np.arange(24)
    plt.figure(figsize=(15, 9))

    for i in range(I):
        plt.plot(hours, generation_data[i], marker='o', linestyle='-', alpha=0.7, label=f'Generator {i}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title("Hourly Electricity Generation for All Generators")
    plt.xticks(hours)  # 0~23 시간 설정
    plt.legend(loc="upper left", fontsize='small')

    plt.show()

def plot_randomized_generation(R, I, T, S):
    hours = np.arange(T)
    
    plt.figure(figsize=(15, 9))

    for i in range(I):
        plt.plot(hours, R[i, :, S], marker='o', linestyle='-', alpha=0.7, label=f'Generator {i}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title(f"Randomized Hourly Generation for Scenario {S}")
    plt.xticks(hours) 
    plt.legend(loc="upper left") 

    plt.show()
       
def plot_scenarios_for_generator(R, i):

    T = R.shape[1]
    S = R.shape[2] 
    hours = np.arange(T) 

    plt.figure(figsize=(15, 9))

    for s in range(S):
        plt.plot(hours, R[i, :, s], linestyle='-', alpha=0.6, label=f'Scenario {s+1}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title(f"Hourly Electricity Generation for Generator {i} Across All Scenarios")
    plt.xticks(hours)
    plt.legend(loc="upper left", fontsize='small', ncol=2)
    plt.show()

def generate_rt_scenarios(rt_da, S, randomness_level):

    rt_da["Time Stamp"] = pd.to_datetime(rt_da["Time Stamp"])
    nyc_rt = rt_da[rt_da["Name"] == "N.Y.C."].copy() 

    nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")
    hourly_avg = nyc_rt.groupby("Hour")["LBMP ($/MWHr)"].mean().reset_index()
    price_hourly = hourly_avg["LBMP ($/MWHr)"].to_numpy()
    T = len(price_hourly)

    np.random.seed(17)
    noise_ranges = {
        "low": (0.95, 1.05),
        "medium": (0.85, 1.15),
        "high": (0.7, 1.3),
    }

    if randomness_level not in noise_ranges:
        raise ValueError("Invalid randomness level. Choose from 'low', 'medium', 'high'.")

    low, high = noise_ranges[randomness_level]
    noise_factors = np.random.uniform(low, high, size=(T, S))

    P_RT = np.expand_dims(price_hourly, axis=-1) * noise_factors

    return P_RT

def plot_rt_scenarios(P_RT):
    T, S = P_RT.shape
    hours = np.arange(T)

    plt.figure(figsize=(15, 8))

    for s in range(S):
        plt.plot(hours, P_RT[:, s], linestyle='-', alpha=0.6, label=f"Scenario {s+1}")

    plt.xlabel("Hour")
    plt.ylabel("Price ($/MWHr)")
    plt.title("Real-Time Price Scenarios (Hourly Averaged)")
    plt.xticks(hours)
    plt.legend(loc="upper left", fontsize="small", ncol=2)

    plt.show()
    
only_profit = np.array(pd.read_csv("result/result_only_profit.csv"))
ny_da = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718da.csv")
ny_rt = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718rt.csv")
ny_da["Time Stamp"] = pd.to_datetime(ny_da["Time Stamp"])
ny_da["Hour"] = ny_da["Time Stamp"].dt.hour
nyc_data = ny_da[ny_da["Name"] == "N.Y.C."]
P_DA = nyc_data["LBMP ($/MWHr)"].to_numpy() * 1.3
P_PN = P_DA * 1.5

# plot_generation_data(generation_data, 10)
# plot_randomized_generation(R, I, T, 7)
# plot_scenarios_for_generator(R, 1)
# plot_rt_scenarios(P_RT)

# include_files = ['1201.csv', '137.csv', '281.csv', '397.csv', '401.csv', '430.csv', '514.csv', '524.csv', '775.csv', '89.csv']
include_files = ['1201.csv', '137.csv', '401.csv', '524.csv', '89.csv']
generation_data, I, T = load_generation_data(include_files, "2022-07-18")

S = 20
randomness_level = "medium"
R = generate_randomized_generation(I, T, S, generation_data, randomness_level)
P_RT = generate_rt_scenarios(ny_rt, S, randomness_level)


✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=20


/var/folders/2_/9m9qjphn0xs45xzpvf1hl0j00000gn/T/ipykernel_64672/2046979868.py:126: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")


### settlement model

In [100]:
set = gp.Model("Settlement")
set.setParam("PoolSolutions", 15)
set.setParam("PoolSearchMode", 2)
set.setParam("PoolGap", 0.05)

x = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
y_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
y_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
M_y = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
z_y = set.addVars(I, T, S, vtype=GRB.BINARY, name="z_y")

d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")
e_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_plus")
e_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_minus")
M_e = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z_e = set.addVars(T, S, vtype=GRB.BINARY, name="z_e") 
set.update()

# obj. func.
obj = gp.quicksum(
    P_DA[t] * x[i, t] for i in range(I) for t in range(T)
) + gp.quicksum(
    1/S * (P_RT[t, s] * e_plus[i, t, s] - P_PN[t] * e_minus[i, t, s])
    for i in range(I) for t in range(T) for s in range(S)
)

set.setObjective(obj, GRB.MAXIMIZE)

Set parameter PoolSolutions to value 15
Set parameter PoolSearchMode to value 2
Set parameter PoolGap to value 0.05


In [101]:
# x로 y+, y- 결정 지어주기
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(R[i, t, s] - x[i, t] == y_plus[i, t, s] - y_minus[i, t, s])

# y+, y- 둘 중 하나는 0이어야함
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(y_plus[i, t, s] <= M_y * z_y[i, t, s])
            set.addConstr(y_minus[i, t, s] <= M_y * (1 - z_y[i, t, s]))

# rt 조건
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(y_plus[i, t, s] <= R[i, t, s])

# sum e+, sum e- 결정 지어주기
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - gp.quicksum(x[i, t] for i in range(I)) 
                          == gp.quicksum(e_plus[i, t, s] for i in range(I)) - gp.quicksum(e_minus[i, t, s] for i in range(I)))

# sum e+, sum e- 둘 중 하나는 0이어야 함
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(gp.quicksum(e_plus[i, t, s] for i in range(I)) <= M_e * z_e[t, s])
            set.addConstr(gp.quicksum(e_minus[i, t, s] for i in range(I)) <= M_e * (1 - z_e[t, s]))

# sum e+ <= sum R
for t in range(T):
    for s in range(S):
        set.addConstr(gp.quicksum(e_plus[i, t, s] for i in range(I)) <= gp.quicksum(R[i, t, s] for i in range(I)))

# e+, e- 정의
for i in range(I):
    for t in range(T):
        for s in range(S):
            # e_plus: 초과량에서 준 양을 뺀 것
            set.addConstr(
                e_plus[i,t,s] == y_plus[i,t,s] - gp.quicksum(d[i,j,t,s] for j in range(I) if j != i)
            )
            # e_minus: 부족량에서 받은 양을 뺀 것
            set.addConstr(
                e_minus[i,t,s] == y_minus[i,t,s] - gp.quicksum(d[j,i,t,s] for j in range(I) if j != i)
            )

# 전력 이동 제한
for i in range(I):
    for t in range(T):
        for s in range(S):
            # 본인이 주는 양은 자신의 y+을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
                <= y_plus[i, t, s]
            )
            # 본인이 받는 양은 자신의 y-을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[j, i, t, s] for j in range(I) if j != i)
                <= y_minus[i, t, s]
            )

# 자기 자신과의 거래 방지
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(d[i, i, t, s] == 0)

In [102]:
set.optimize()

if set.status == GRB.OPTIMAL:
    x_vals = np.array([[x[i, t].x for t in range(T)] for i in range(I)])
    y_plus_vals = np.array(
        [[[y_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    y_minus_vals = np.array(
        [[[y_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    d_vals = np.array(
        [
            [[[d[i, j, t, s].x for s in range(S)] for t in range(T)] for j in range(I)]
            for i in range(I)
        ]
    )
    e_plus_vals = np.array(
        [[[e_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    e_minus_vals = np.array(
        [[[e_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    given_vals = np.array(
        [[[sum(d[i, j, t, s].x for j in range(I) if j != i) for s in range(S)] for t in range(T)] for i in range(I)]
    )
    received_vals = np.array(
        [[[sum(d[j, i, t, s].x for j in range(I) if j != i) for s in range(S)] for t in range(T)] for i in range(I)]
    )
    print("\n- - - - - - - - - - - - - - - - - - - - - - - - -")
    print("Optimal solution found!")
    print(set.objVal)
    print(f"Optimality Gap: {set.MIPGap}")
else:
    print("\nNo optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
PoolSolutions  15
PoolSearchMode  2
PoolGap  0.05

Optimize a model with 29280 rows, 24600 columns and 141600 nonzeros
Model fingerprint: 0x91d4307b
Variable types: 21720 continuous, 2880 integer (2880 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 2e+03]
Presolve removed 21140 rows and 11160 columns
Presolve time: 0.05s
Presolved: 8140 rows, 13440 columns, 38240 nonzeros
Variable types: 10560 continuous, 2880 integer (2880 binary)
Found heuristic solution: objective 947501.79666

Root relaxation: objective 1.554658e+06, 5008 iterations, 0.05 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI

In [103]:
# print(f"Status: {set.status}")
# if set.status == GRB.OPTIMAL:
#     print("모델이 Optimal입니다")

#     # Solution pool 확인
#     print("\nSolution Pool 정보:")
#     print(f"찾은 해의 개수: {set.SolCount}")

#     # Best solution의 objective value
#     best_obj = set.ObjVal
#     print(f"\nBest Solution의 Objective Value: {best_obj}")

#     # 각 솔루션의 목적함수 값 출력 및 best solution 여부 확인
#     solutions = []
#     for i in range(set.SolCount):
#         set.params.SolutionNumber = i
#         solutions.append((i, set.PoolObjVal))
#         print(f"\nSolution {i}:")
#         print(f"Objective value: {set.PoolObjVal}")
#         print(f"Objective bound: {set.PoolObjBound}")
#         print(
#             f"Is Best Solution: {abs(set.PoolObjVal - best_obj) < 1e-9}"
#         )  # 수치 오차 고려

#         # Gap 계산 (best solution에 대해서만)
#         if abs(set.PoolObjVal - best_obj) < 1e-9:
#             gap = abs(set.PoolObjVal - set.ObjBound) / abs(set.PoolObjVal)
#             print(f"Gap: {gap * 100:.3f}%")

#     # Solution pool 파라미터 확인
#     print("\nSolution Pool 파라미터:")
#     print(f"PoolSolutions: {set.Params.PoolSolutions}")
#     print(f"PoolSearchMode: {set.Params.PoolSearchMode}")
#     print(f"PoolGap: {set.Params.PoolGap}")
# elif set.status == GRB.UNBOUNDED:
#     print("모델이 Unbounded입니다")
#     # Unbounded ray 확인
#     set.computeIIS()
#     set.write("model.ilp")
# elif set.status == GRB.INFEASIBLE:
#     print("모델이 Infeasible입니다")
#     # IIS 계산
#     set.computeIIS()
#     set.write("model.ilp")
# elif set.status == GRB.INF_OR_UNBD:
#     print("모델이 Infeasible 또는 Unbounded입니다")
#     # Presolve를 끄고 다시 시도
#     set.setParam('Presolve', 0)
#     set.optimize()
# elif set.status == GRB.TIME_LIMIT:
#     print("시간 제한에 도달했습니다")
#     if set.SolCount > 0:
#         print(f"현재까지의 최선해: {set.objVal}")
#         print(f"Optimality Gap: {set.MIPGap}")

In [104]:
# solution_count = set.SolCount
# print(f"총 {solution_count}개의 해가 저장되었습니다.")

# for i in range(solution_count):
#     set.setParam(GRB.Param.SolutionNumber, i)
#     print(f"\n해 {i+1}: 목적 함수 값 = {set.PoolObjVal}")
#     for i in range(0,3):
#         for t in range(13,14): 
#             print(f"x[{i},{t}] = {x[i,t].Xn}")
#     for s in range(0,5):
#         for t in range(13,14):
#             for i in range(0,1):
#                 for j in range(5,10):
#                     print(f"d[{i},{j},{t},{s}] = {d[i, j, t, s].Xn}")

### 결과 분석

#### 수익 분석

In [127]:
# 시간별 Day-ahead 수익 계산
hourly_da = np.zeros(T)
for t in range(T):
    hourly_da[t] = sum(P_DA[t] * x[i, t].x for i in range(I))

# 전체 Day-ahead 수익
daily_da = np.sum(hourly_da)


# 시간별 Real-time 수익 계산
hourly_rt = np.zeros(T)
for t in range(T):
    hourly_rt[t] = sum(
        (1 / S) * sum(P_RT[t, s] * e_plus[i, t, s].x for s in range(S)) for i in range(I)
    )

# 전체 Real-time 수익
daily_rt = np.sum(hourly_rt)


# 시간별 패널티 비용 계산
hourly_pen = np.zeros(T)
for t in range(T):
    hourly_pen[t] = sum(
        (1 / S) * sum(P_PN[t] * e_minus[i, t, s].x for s in range(S)) for i in range(I)
    )

# 전체 패널티 비용
daily_pen = np.sum(hourly_pen)


# 시간별 총 시스템 이익 계산
hourly_system_profit = hourly_da + hourly_rt - hourly_pen

# 전체 총 시스템 이익
daily_system_profit = daily_da + daily_rt - daily_pen

# 📌 결과 출력
print("🔹 Hourly Profits (by t):")
for t in range(T):
    print(f"[t={t:02d}] DA: {hourly_da[t]:.2f}, RT: {hourly_rt[t]:.2f}, "
          f"Penalty: {hourly_pen[t]:.2f}, Total Profit: {hourly_system_profit[t]:.2f}")

print("\n🔹 Total Profits (summed over all t):")
print(f"Total DA Profit: {daily_da:.2f}")
print(f"Total RT Profit: {daily_rt:.2f}")
print(f"Total Penalty Cost: {daily_pen:.2f}")
print(f"Total System Profit: {daily_system_profit:.2f}")

🔹 Hourly Profits (by t):
[t=00] DA: 0.00, RT: 0.00, Penalty: 0.00, Total Profit: 0.00
[t=01] DA: 0.00, RT: 0.00, Penalty: 0.00, Total Profit: 0.00
[t=02] DA: 0.00, RT: 0.00, Penalty: 0.00, Total Profit: 0.00
[t=03] DA: 0.00, RT: 0.00, Penalty: 0.00, Total Profit: 0.00
[t=04] DA: 21.82, RT: 14.77, Penalty: 0.13, Total Profit: 36.46
[t=05] DA: 0.00, RT: 0.00, Penalty: 0.00, Total Profit: 0.00
[t=06] DA: 244.53, RT: 34.53, Penalty: 6.79, Total Profit: 272.27
[t=07] DA: 910.96, RT: 203.72, Penalty: 12.32, Total Profit: 1102.35
[t=08] DA: 3291.18, RT: 1369.30, Penalty: 6.82, Total Profit: 4653.66
[t=09] DA: 20180.45, RT: 1718.91, Penalty: 1432.64, Total Profit: 20466.72
[t=10] DA: 45678.85, RT: 10968.53, Penalty: 485.30, Total Profit: 56162.07
[t=11] DA: 0.00, RT: 83501.39, Penalty: 0.00, Total Profit: 83501.39
[t=12] DA: 0.00, RT: 152715.07, Penalty: 0.00, Total Profit: 152715.07
[t=13] DA: 0.00, RT: 349853.57, Penalty: 0.00, Total Profit: 349853.57
[t=14] DA: 187134.22, RT: 18867.34, Pena

#### 하루 aggregated 커밋량 분석

In [106]:
for t in range(T):
    avg_e_plus = sum(e_plus_vals[i, t, s] for i in range(I) for s in range(S)) / S
    avg_e_minus = sum(e_minus_vals[i, t, s] for i in range(I) for s in range(S)) / S
    print(f"[시간 {t}] x: {sum(x_vals[i, t] for i in range(I)):.3f}, e+: {avg_e_plus:.3f}, e-: {avg_e_minus:.3f}")

total_x = sum(x_vals[i, t] for i in range(I) for t in range(T))
print(f"총 하루 commitment: {total_x:.3f}")

[시간 0] x: 0.000, e+: 0.000, e-: 0.000
[시간 1] x: 0.000, e+: 0.000, e-: 0.000
[시간 2] x: 0.000, e+: 0.000, e-: 0.000
[시간 3] x: 0.000, e+: 0.000, e-: 0.000
[시간 4] x: 0.284, e+: 0.202, e-: 0.001
[시간 5] x: 0.000, e+: 0.000, e-: 0.000
[시간 6] x: 2.815, e+: 0.470, e-: 0.052
[시간 7] x: 9.587, e+: 2.286, e-: 0.086
[시간 8] x: 29.023, e+: 12.938, e-: 0.040
[시간 9] x: 164.460, e+: 17.233, e-: 7.784
[시간 10] x: 359.906, e+: 93.423, e-: 2.549
[시간 11] x: 0.000, e+: 614.653, e-: 0.000
[시간 12] x: 0.000, e+: 838.797, e-: 0.000
[시간 13] x: 0.000, e+: 1323.659, e-: 0.000
[시간 14] x: 1184.476, e+: 149.972, e-: 10.423
[시간 15] x: 0.000, e+: 801.951, e-: 0.000
[시간 16] x: 605.583, e+: 136.019, e-: 7.912
[시간 17] x: 0.000, e+: 790.294, e-: 0.000
[시간 18] x: 0.000, e+: 516.846, e-: 0.000
[시간 19] x: 351.925, e+: 51.885, e-: 3.326
[시간 20] x: 100.590, e+: 12.278, e-: 3.414
[시간 21] x: 20.093, e+: 3.235, e-: 1.117
[시간 22] x: 0.000, e+: 0.000, e-: 0.000
[시간 23] x: 0.000, e+: 0.000, e-: 0.000
총 하루 commitment: 2828.742


#### exchange process

In [107]:
for t in range(19, 22):
    x_sum = sum(x_vals[i, t] for i in range(I))

    for s in range(1,4):
        y_plus_sum = sum(y_plus_vals[i, t, s] for i in range(I))
        y_minus_sum = sum(y_minus_vals[i, t, s] for i in range(I))

        e_plus_sum = sum(e_plus_vals[i, t, s] for i in range(I))
        e_minus_sum = sum(e_minus_vals[i, t, s] for i in range(I))

        R_sum = sum(R[i, t, s] for i in range(I))

        print(f"[t={t} s={s}]")

        print(f"x 합계: {x_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {x_vals[i,t]:.2f}", end=" ")
        print(")")

        print(f"R 합계: {R_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {R[i, t, s]:.2f}", end=" ")
        print(")")

        print(f"y+ 합계: {y_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"y- 합계: {y_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"e+ 합계: {e_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"e- 합계: {e_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print("거래 내역:")
        for i in range(I):
            for j in range(I):
                if d_vals[i, j, t, s] > 1e-6:
                    print(
                        f"발전기 {i}가 발전기 {j}에게 {d_vals[i,j,t,s]:.2f} 전력을 줌"
                    )
        print()

[t=19 s=1]
x 합계: 351.92 ([0] 54.49 [1] 53.45 [2] 39.01 [3] 158.91 [4] 46.06 )
R 합계: 448.97 ([0] 43.85 [1] 76.65 [2] 27.91 [3] 272.08 [4] 28.47 )
y+ 합계: 136.37 ([0] 0.00 [1] 23.21 [2] 0.00 [3] 113.17 [4] 0.00 )
y- 합계: 39.33 ([0] 10.64 [1] 0.00 [2] 11.10 [3] 0.00 [4] 17.59 )
e+ 합계: 97.04 ([0] 0.00 [1] 12.57 [2] 0.00 [3] 84.48 [4] 0.00 )
e- 합계: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 )
거래 내역:
발전기 1가 발전기 0에게 10.64 전력을 줌
발전기 3가 발전기 2에게 11.10 전력을 줌
발전기 3가 발전기 4에게 17.59 전력을 줌

[t=19 s=2]
x 합계: 351.92 ([0] 54.49 [1] 53.45 [2] 39.01 [3] 158.91 [4] 46.06 )
R 합계: 406.32 ([0] 54.49 [1] 40.39 [2] 55.20 [3] 231.41 [4] 24.83 )
y+ 합계: 88.69 ([0] 0.00 [1] 0.00 [2] 16.19 [3] 72.50 [4] 0.00 )
y- 합계: 34.29 ([0] 0.00 [1] 13.05 [2] 0.00 [3] 0.00 [4] 21.24 )
e+ 합계: 54.40 ([0] 0.00 [1] 0.00 [2] 3.14 [3] 51.26 [4] 0.00 )
e- 합계: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 )
거래 내역:
발전기 2가 발전기 1에게 13.05 전력을 줌
발전기 3가 발전기 4에게 21.24 전력을 줌

[t=19 s=3]
x 합계: 351.92 ([0] 54.49 [1] 53.45 [2] 39.01 [3] 

### 정산

In [108]:
def hourly_contribution(data):
    
    if len(data.shape) == 2:  # Case 1: [i,t]
        I, T = data.shape
        normalized_data = np.zeros((I, T))

        for t in range(T):
            total = np.sum(data[:, t])  # 각 시간별 총 발전량
            if total > 0:
                normalized_data[:, t] = data[:, t] / total  # 비율 계산
            else:
                normalized_data[:, t] = 0  # 발전량이 0이면 0으로 설정
                
    elif len(data.shape) == 3:  # Case 2: [i,t,s]
        I, T, S = data.shape
        normalized_data = np.zeros((I, T))

        for t in range(T):
            scenario_mean = np.mean(data[:, t, :], axis=1)  # 각 발전기의 시나리오 평균
            total = np.sum(scenario_mean)  # 각 시간별 총 발전량 (시나리오 평균 기준)
            
            if total > 0:
                normalized_data[:, t] = scenario_mean / total  # 비율 계산
            else:
                normalized_data[:, t] = 0  # 발전량이 0이면 0으로 설정

    else:
        raise ValueError("Input data must be of shape (I, T) or (I, T, S).")
    
    return normalized_data

def daily_contribution(data):
    
    if len(data.shape) == 2:  # Case 1: [i,t]
        I, T = data.shape
        daily_total = np.sum(data)  # 하루 동안 전체 발전량
        normalized_data = np.zeros(I)

        if daily_total > 0:
            normalized_data = np.sum(data, axis=1) / daily_total  # 하루 기여도 계산
        else:
            normalized_data[:] = 0  # 발전량이 0이면 0으로 설정

    elif len(data.shape) == 3:  # Case 2: [i,t,s]
        I, T, S = data.shape
        normalized_data = np.zeros(I)

        scenario_mean = np.mean(data, axis=2)  # 시나리오 평균 계산 (I, T)
        daily_total = np.sum(scenario_mean)  # 하루 동안 전체 발전량 (시나리오 평균 기준)
        
        if daily_total > 0:
            normalized_data = np.sum(scenario_mean, axis=1) / daily_total  # 하루 기여도 계산
        else:
            normalized_data[:] = 0  # 발전량이 0이면 0으로 설정

    else:
        raise ValueError("Input data must be of shape (I, T) or (I, T, S).")
    
    return normalized_data

def remuneration(contribution, amount):
    
    # **Case 1: 시간별 정산**
    if len(contribution.shape) == 2 and len(amount.shape) == 1:
        I, T = contribution.shape
        if amount.shape[0] != T:
            raise ValueError("Hourly amount (T,) should match contribution shape (I, T).")

        # 시간별 정산액 = 시간별 기여도 * 시간별 총 분배금액
        remuneration_amount = contribution * amount.reshape(1, T)  # (I, T)

        # 각 발전기의 총 정산액 (t에 대한 합)
        total_remuneration = np.sum(remuneration_amount, axis=1)  # (I,)

    # **Case 2: 하루 단위 정산**
    elif len(contribution.shape) == 1 and len(amount.shape) == 0:
        I = contribution.shape[0]

        # 하루 단위 정산액 = 하루 기여도 * 하루 총 분배금액
        remuneration_amount = contribution * amount  # (I,)

        # 하루 단위 정산에서는 총합이 원래 하루 정산액과 동일
        total_remuneration = remuneration_amount.copy()  # (I,)

    else:
        raise ValueError("Invalid input shapes. Expected (I,T) with (T,) or (I,) with (1,).")

    return remuneration_amount, total_remuneration

In [144]:
print(pd.DataFrame(hourly_contribution(x_vals)))
print(pd.DataFrame(daily_contribution(x_vals)))

remuneration_hourly, hourly_total = remuneration(hourly_contribution(x_vals), hourly_system_profit)
remuneration_daily, daily_total = remuneration(daily_contribution(x_vals), daily_system_profit)

print(pd.DataFrame(remuneration_hourly))
print(pd.DataFrame(hourly_total))
print(pd.DataFrame(remuneration_daily))

     0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18    19    20    21    22    23
0 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.189 0.029 0.129 0.000 0.000 0.000 0.513 0.000 0.046 0.000 0.000 0.155 0.342 0.435 0.000 0.000
1 0.000 0.000 0.000 0.000 0.000 0.000 0.231 0.328 0.239 0.278 0.150 0.000 0.000 0.000 0.487 0.000 0.000 0.000 0.000 0.152 0.140 0.000 0.000 0.000
2 0.000 0.000 0.000 0.000 1.000 0.000 0.769 0.328 0.358 0.397 0.380 0.000 0.000 0.000 0.000 0.000 0.094 0.000 0.000 0.111 0.106 0.035 0.000 0.000
3 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.038 0.132 0.160 0.174 0.000 0.000 0.000 0.000 0.000 0.780 0.000 0.000 0.452 0.323 0.530 0.000 0.000
4 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.305 0.082 0.135 0.167 0.000 0.000 0.000 0.000 0.000 0.080 0.000 0.000 0.131 0.090 0.000 0.000 0.000
      0
0 0.279
1 0.267
2 0.115
3 0.271
4 0.068
     0     1     2     3      4     5       6       7        8        9     

### 결과 저장

In [110]:
# set_profit_value = pd.DataFrame({'set_profit_value': der_profit})
# set_profit_value.to_csv('result/result_set1_profit.csv', index=False)

# set_obj = pd.DataFrame({'set_obj': [set.objVal]})
# set_obj.to_csv('result/result_set1_obj.csv', index=False)